In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import numpy as np
import cv2
import random

tf.logging.set_verbosity(tf.logging.INFO)

Using TensorFlow backend.


In [2]:
# There are only labels in the dataset for the last image in a sequence, but the few before that are 
# still quite expressive. Including them will greatly augment our dataset. 
# The images need to be moved so keras' generators will work

def move_tagged_images():
    train_path = os.path.join("aligned", "train")
    valid_path = os.path.join("aligned", "valid")
    path = ""
    training = 0
    valid = 0
    
    for path, name, files in os.walk('Emotion'):

        for file in files:
            if(file):
                with open(os.path.join(path, file)) as curf:
                    n = int(file[-14:-12])
                    print(n)
                    
                    score = int(curf.read().strip(' ')[:1]) + 1
                    print("score", score)
                    
                    for i in range(4):
                        fi = file[:-14] + str(n - i).zfill(2) + ".png"
                        
                        if(random.randint(1, 10) < 8):
                            path = train_path
                            training = training + 1
                        else:
                            path = valid_path
                            valid = valid + 1
                        
                        print("moved", os.path.join("aligned", fi), "to", os.path.join(path, str(score), fi))
                        try:
                            os.rename(os.path.join("aligned", fi), os.path.join(path, str(score), fi))
                        except:
                            pass
                        
    print("training, valid", training, valid)
    
                    

In [2]:
train_dir = os.path.join("aligned", "train")
valid_dir = os.path.join("aligned", "valid")
batch_size = 2
nb_train = 931
nb_valid = 381
img_w, img_h = 340, 340
steps_per_epoch = nb_train // batch_size
valid_steps = nb_valid // batch_size
epochs = 100

In [3]:

train_datagen = ImageDataGenerator(
                rescale = 1./255,
                horizontal_flip = True,
                rotation_range = 20)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
train_data = train_datagen.flow_from_directory(train_dir, batch_size = batch_size, target_size = (340, 340), class_mode="categorical")
valid_data = test_datagen.flow_from_directory(valid_dir, batch_size = batch_size, target_size = (340, 340), class_mode="categorical")

Found 931 images belonging to 8 classes.
Found 381 images belonging to 8 classes.


In [5]:
print(dir(train_data))
print(train_data.image_shape)
print(train_data.class_indices)
print(train_data.data_format)
print(train_data.class_mode)
print(train_data.target_size)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_flow_index', 'batch_index', 'batch_size', 'class_indices', 'class_mode', 'classes', 'color_mode', 'data_format', 'directory', 'filenames', 'image_data_generator', 'image_shape', 'index_generator', 'lock', 'n', 'next', 'num_class', 'reset', 'samples', 'save_format', 'save_prefix', 'save_to_dir', 'shuffle', 'target_size', 'total_batches_seen']
(340, 340, 3)
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}
channels_last
categorical
(340, 340)


In [6]:
m = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=[340, 340, 3], pooling=max)

for layer in m.layers:
    layer.trainable = False
    
x = m.output

#x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(8, activation='softmax')(x)

model = keras.models.Model(inputs=m.input, outputs = x)
model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 340, 340, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 340, 340, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 340, 340, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 170, 170, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 170, 170, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 170, 170, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 85, 85, 128)       0         
__________

In [7]:
model.fit_generator(train_data, 
                    steps_per_epoch = steps_per_epoch, 
                    epochs = epochs, 
                    validation_data = valid_data, 
                    validation_steps = valid_steps)
model.save('second_try.h5')

Epoch 1/100
465/465 [==============================] - 36s - loss: 12.1045 - acc: 0.2473 - val_loss: 11.8341 - val_acc: 0.2658
Epoch 2/100
465/465 [==============================] - 36s - loss: 12.1146 - acc: 0.2484 - val_loss: 11.9078 - val_acc: 0.2612
Epoch 3/100
465/465 [==============================] - 35s - loss: 12.1146 - acc: 0.2484 - val_loss: 11.6527 - val_acc: 0.2770
Epoch 4/100
465/465 [==============================] - 35s - loss: 12.1492 - acc: 0.2462 - val_loss: 11.6952 - val_acc: 0.2744
Epoch 5/100
465/465 [==============================] - 35s - loss: 12.1146 - acc: 0.2484 - val_loss: 11.9504 - val_acc: 0.2586
Epoch 6/100
465/465 [==============================] - 36s - loss: 12.1666 - acc: 0.2452 - val_loss: 11.6101 - val_acc: 0.2797
Epoch 7/100
465/465 [==============================] - 36s - loss: 12.0279 - acc: 0.2538 - val_loss: 11.9929 - val_acc: 0.2559
Epoch 8/100
465/465 [==============================] - 36s - loss: 12.1666 - acc: 0.2452 - val_loss: 11.9078 - 

465/465 [==============================] - 37s - loss: 12.0452 - acc: 0.2527 - val_loss: 11.8228 - val_acc: 0.2665
Epoch 66/100
465/465 [==============================] - 37s - loss: 12.0799 - acc: 0.2505 - val_loss: 11.6527 - val_acc: 0.2770
Epoch 67/100
465/465 [==============================] - 37s - loss: 12.1493 - acc: 0.2462 - val_loss: 11.7802 - val_acc: 0.2691
Epoch 68/100
465/465 [==============================] - 36s - loss: 12.2359 - acc: 0.2409 - val_loss: 12.1630 - val_acc: 0.2454
Epoch 69/100
465/465 [==============================] - 36s - loss: 11.8546 - acc: 0.2645 - val_loss: 12.1205 - val_acc: 0.2480
Epoch 70/100
465/465 [==============================] - 36s - loss: 12.2359 - acc: 0.2409 - val_loss: 11.6101 - val_acc: 0.2797
Epoch 71/100
465/465 [==============================] - 36s - loss: 12.0626 - acc: 0.2516 - val_loss: 12.0779 - val_acc: 0.2507
Epoch 72/100
465/465 [==============================] - 37s - loss: 12.4265 - acc: 0.2290 - val_loss: 11.5251 - val_a